# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "human_enhancers_cohn"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

human_enhancers_cohn 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

2021-11-23 01:07:13.685202: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-23 01:07:13.685227: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


2021-11-23 01:07:17.682703: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-11-23 01:07:17.682742: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michle): /proc/driver/nvidia/version does not exist
2021-11-23 01:07:17.683624: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
info(DATASET)

/home/petr/projects/genomic_benchmarks/src/genomic_benchmarks/utils/datasets.py:50: UserWarning: No version specified. Using version 0.
  warnings.warn(f"No version specified. Using version {metadata['version']}.")


Dataset `human_enhancers_cohn` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 500.

Totally 27791 sequences have been found, 20843 for training and 6948 for testing.


,train,test
negative,10422,3474
positive,10421,3474


## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 20843 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
# vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

2021-11-23 01:07:22.146862: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


['', '[UNK]', 't', 'a', 'c', 'g']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10


  1/326 [..............................] - ETA: 7:12 - loss: 0.7662 - binary_accuracy: 0.4375 - f1_score: 0.0000e+00

  3/326 [..............................] - ETA: 10s - loss: 0.7419 - binary_accuracy: 0.4323 - f1_score: 0.0000e+00 

  5/326 [..............................] - ETA: 10s - loss: 0.7074 - binary_accuracy: 0.5188 - f1_score: 0.0235    

  7/326 [..............................] - ETA: 10s - loss: 0.6898 - binary_accuracy: 0.5558 - f1_score: 0.0253

...

319/326 [============================>.] - ETA: 0s - loss: 0.4812 - binary_accuracy: 0.7640 - f1_score: 0.7177

321/326 [============================>.] - ETA: 0s - loss: 0.4811 - binary_accuracy: 0.7637 - f1_score: 0.7176

323/326 [============================>.] - ETA: 0s - loss: 0.4810 - binary_accuracy: 0.7638 - f1_score: 0.7174

325/326 [============================>.] - ETA: 0s - loss: 0.4806 - binary_accuracy: 0.7639 - f1_score: 0.7177

326/326 [==============================] - 13s 40ms/step - loss: 0.4807 - binary_accuracy: 0.7638 - f1_score: 0.7176


## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 6948 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

  1/109 [..............................] - ETA: 40s - loss: 0.5929 - binary_accuracy: 0.7188 - f1_score: 0.7895

  5/109 [>.............................] - ETA: 1s - loss: 0.5896 - binary_accuracy: 0.7188 - f1_score: 0.7855 

  8/109 [=>............................] - ETA: 1s - loss: 0.6328 - binary_accuracy: 0.6934 - f1_score: 0.7677

 11/109 [==>...........................] - ETA: 1s - loss: 0.6731 - binary_accuracy: 0.6818 - f1_score: 0.7610

 13/109 [==>...........................] - ETA: 2s - loss: 0.6984 - binary_accuracy: 0.6743 - f1_score: 0.7556

...

100/109 [==========================>...] - ETA: 0s - loss: 0.7593 - binary_accuracy: 0.6652 - f1_score: 0.7418

103/109 [===========================>..] - ETA: 0s - loss: 0.7598 - binary_accuracy: 0.6652 - f1_score: 0.7419

107/109 [============================>.] - ETA: 0s - loss: 0.7609 - binary_accuracy: 0.6662 - f1_score: 0.7430

109/109 [==============================] - 2s 18ms/step - loss: 0.7626 - binary_accuracy: 0.6655 - f1_score: 0.7426


[0.762627899646759, 0.6655152440071106, 0.7426068186759949]